# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [1]:
!pip3 install transformers==4.2.2
!pip install trinket
!pip install torch==1.7.0
!git clone https://github.com/NVIDIA/apex


     |████████████████████████████████| 1.8 MB 10.1 MB/s 
     |████████████████████████████████| 895 kB 83.6 MB/s 
     |████████████████████████████████| 2.9 MB 82.2 MB/s 
     |████████████████████████████████| 71 kB 342 kB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
     |████████████████████████████████| 104 kB 43.1 MB/s 
     |████████████████████████████████| 58 kB 8.7 MB/s 
     |████████████████████████████████| 482 kB 57.9 MB/s 
     |████████████████████████████████| 54 kB 4.2 MB/s 
  Created wheel for trinket: filename=trinket-0.1.5-py3-none-any.whl size=17190 sha256=d3d3bba0e5cbec8c3dc9c0079ad4748a4f3be0a84783790712bfac403d760303
  Stored in directory: /root/.cache/pip/wheels/4b/3d/8c/3626166497e28fa61399f8b07abb5b8edd8b254d429b5e0ad9
  Created wheel for autoroutes: filename=autoroutes-0.2.1-cp37-cp37m-linux_x86_64.whl size=197125 sha256=119844f00274afe2fb3889eb355f962f6c8d2be7c195c450cb9646bd2617b678
  Stored in directory: /root/.cache/pip/wheels/a5/13/d4/d

In [ ]:
cd apex

In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir \
--global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

## Download files

In [ ]:
!pip install dvach.api

     |████████████████████████████████| 81 kB 5.6 MB/s 
     |████████████████████████████████| 57 kB 6.6 MB/s 
     |████████████████████████████████| 58 kB 7.6 MB/s 
  Created wheel for addict: filename=addict-2.2.0-py3-none-any.whl size=2783 sha256=e91e13bf8ca93c4f295a271d68b4c6d4b064e0b4c1dbe0291c2b4c40931dbc9d
  Stored in directory: /root/.cache/pip/wheels/21/92/be/96ccf19dabf01038b695fef37b457017a3a8811a7d36f27ef3
  Created wheel for simplejson: filename=simplejson-3.16.0-cp37-cp37m-linux_x86_64.whl size=114888 sha256=7bd44467e0fcc9f299c0d23176f9741211c78d7a58631907d6f09302d52233bf
  Stored in directory: /root/.cache/pip/wheels/00/6f/29/109ee3b85a9f9861d4294b05e4597363d5ad3c9dfaccc78787
Successfully built addict simplejson
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
    

In [ ]:
import api2ch
brds= ['b','d','sn','cc','soc','po','un','o','po']
ch = []
fieldnames = ['board','post','tags']
import csv
def parse_2ch():
  from tqdm import tqdm, trange
  with open('2ch.csv', 'w', newline='') as csvfile:
          
          writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
          writer.writeheader()
          for b in brds:

            api = api2ch.DvachApi(b)

            board = api.get_board()
            pbar = tqdm(total=len(board))

            print(f'Total {len(board)} threads in {api.board.id}')
            for thread in board:
                #print(thread)
                pbar.update(1)
                try:
                  thread = api.get_thread(thread)
                  pbar1 = tqdm(total=len(thread))

                  for post in thread:
                    pbar1.update(1)
                    #print(post.comment)
                    try:
                      comm = post.comment.split('</a><br>')[1]
                      tags = post.tags
                      writer.writerow({'board':b, 'post':comm,'tags':tags  })
                    except:
                      #print(post.comment)
                      writer.writerow({'board':b, 'post':post.comment,'tags':tags  })
                  #ch+=[(thread.post.comment)]
                  pbar1.close()
                except:
                    print('err')
            pbar.close()  


In [2]:
!gdown https://drive.google.com/uc?id=1sYdutKVcXgcg-lTtanv1WkGQ4cTYNz0d

Downloading...
From: https://drive.google.com/uc?id=1sYdutKVcXgcg-lTtanv1WkGQ4cTYNz0d
To: /content/2ch.csv
100% 31.4M/31.4M [00:00<00:00, 86.3MB/s]


In [3]:
import pandas as pd
df = pd.read_csv('2ch.csv')

In [5]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="wch", sort=None)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
df.head()

,board,post,tags
0,b,Таблицы пищевой ценности и химического состава...,NaN
1,b,Бамп,NaN
2,b,Где купить без рецепта?,{}
3,b,"Я в авиценна-фарма купил последний раз, Голдла...",{}
4,b,"<span class=""unkfunc"">&gt;Пик4</span><br><br>З...",{}


In [7]:
df = df.dropna()

In [8]:
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [9]:
posts = []
for post in df.post:
  
  if post:
    if len(post)>5:

      post = strip_tags(post)
      post = post.replace('br','').replace('<span class="spoiler">','').replace('</strong>','').replace('<strong>','').replace('<span class="unkfunc">&g','')
      posts+=[post]

In [10]:
df = df.drop_duplicates()
df = df.dropna()

In [11]:
posts

['Где купить без рецепта?',
 'Я в авиценна-фарма купил последний раз, Голдлайн Плюс покупай, потому что он продается не по строгому 148 учетному рецепту как просто Голдлайн, хотя по факту это один и тот же препаратhttps://www.unico94.ru/consult/61669/',
 '>Пик4Застрял между 15 и 20%, где-то 17 по прикидкам, и сука ни в какую. С 28% слез оче быстро, на этих значениях пердолюсь уже полгода без особых визуальных изменений, хотя дефицит вроде есть. Ну качаюсь помаленьку с гантелькой, мышцы оформились, на руках рельеф нарисовался, но прослойка на квадратах никак не слазит. Чяднт?',
 'Откуда у тебя рецепт на сибутрамин? Все аптеки нахуй шлют когда пытаешься купить.',
 '>нет белковХорошая диета, правильная, неча сохранять работоспособность - от лукавого это, и думалка тоже не нужна, вон деды на гречке жили...',
 '>С 28% слез оче быстро, на этих значениях пердолюсь уже полгода без особых визуальных изменений, хотя дефицит вроде есть. Ну качаюсь помаленьку с гантелькой, мышцы оформились, на рук

In [12]:
df = pd.DataFrame()

In [13]:
df['line']= posts

In [14]:
df.to_csv('posts.csv')

In [15]:
all_rick = pd.read_csv('posts.csv')
all_rick.head(10)

,Unnamed: 0,line
0,0,Где купить без рецепта?
1,1,"Я в авиценна-фарма купил последний раз, Голдла..."
2,2,">Пик4Застрял между 15 и 20%, где-то 17 по прик..."
3,3,Откуда у тебя рецепт на сибутрамин? Все аптеки...
4,4,">нет белковХорошая диета, правильная, неча сох..."
5,5,">С 28% слез оче быстро, на этих значениях перд..."
6,6,"Мне кажется, ОП - жирный тролль"
7,7,">>нет белков>Хорошая диета, правильная, неча с..."
8,8,">Мне кажется, ОП - жирный тролльС хуя ли? Я са..."
9,9,Греча писча богов!


In [16]:
torch.__version__

NameError: ignored

In [17]:
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)
from torch.cuda.amp import autocast

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [18]:
# Args to allow for easy convertion of python script to notebook
backbone = 'sberbank-ai/rugpt3medium_based_on_gpt2'
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = backbone
        self.config_name = backbone
        self.tokenizer_name = backbone
        self.cache_dir = 'cached'
        self.block_size = 16
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 1
        self.per_gpu_eval_batch_size = 1
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 7000
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = False
        self.overwrite_cache = False
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = True
        self.fp16_opt_level = 'O3'

args = Args()

In [19]:
args.n_gp

AttributeError: ignored

In [19]:
df = pd.read_csv('/content/posts.csv')

In [20]:
df

,Unnamed: 0,line
0,0,Где купить без рецепта?
1,1,"Я в авиценна-фарма купил последний раз, Голдла..."
2,2,">Пик4Застрял между 15 и 20%, где-то 17 по прик..."
3,3,Откуда у тебя рецепт на сибутрамин? Все аптеки...
4,4,">нет белковХорошая диета, правильная, неча сох..."
...,...,...
93942,93942,Какая граната у хохлов? У них нихуя нет.
93943,93943,> Подмена мочи на ОлимпиадеЗа которую русня из...
93944,93944,>>сбитый Боинг>Никаких пруфовНЕСКОЛЬКО ЛЕТ РАС...
93945,93945,>РАССМАТРИВАЕТ МЕЖДУНАРОДНЫЙ ЧЕСТНЫЙ СУДтипа а...


In [21]:
contexted = []

n = 7

for i in range(n, len(df['line'])):
  row = []
  prev = i - 1 - n # we additionally substract 1, so row will contain current responce and 7 previous responces  
  for j in range(i, prev, -1):
    row.append(df['line'][j])
  contexted.append(row)  


In [22]:
contexted = contexted[:(len(contexted)//2)]

In [23]:
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(n-1)]
columns

['response',
 'context',
 'context/0',
 'context/1',
 'context/2',
 'context/3',
 'context/4',
 'context/5']

In [24]:
df = pd.DataFrame.from_records(contexted, columns=columns)
df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,">>нет белков>Хорошая диета, правильная, неча с...","Мне кажется, ОП - жирный тролль",">С 28% слез оче быстро, на этих значениях перд...",">нет белковХорошая диета, правильная, неча сох...",Откуда у тебя рецепт на сибутрамин? Все аптеки...,">Пик4Застрял между 15 и 20%, где-то 17 по прик...","Я в авиценна-фарма купил последний раз, Голдла...",Где купить без рецепта?
1,">Мне кажется, ОП - жирный тролльС хуя ли? Я са...",">>нет белков>Хорошая диета, правильная, неча с...","Мне кажется, ОП - жирный тролль",">С 28% слез оче быстро, на этих значениях перд...",">нет белковХорошая диета, правильная, неча сох...",Откуда у тебя рецепт на сибутрамин? Все аптеки...,">Пик4Застрял между 15 и 20%, где-то 17 по прик...","Я в авиценна-фарма купил последний раз, Голдла..."
2,Греча писча богов!,">Мне кажется, ОП - жирный тролльС хуя ли? Я са...",">>нет белков>Хорошая диета, правильная, неча с...","Мне кажется, ОП - жирный тролль",">С 28% слез оче быстро, на этих значениях перд...",">нет белковХорошая диета, правильная, неча сох...",Откуда у тебя рецепт на сибутрамин? Все аптеки...,">Пик4Застрял между 15 и 20%, где-то 17 по прик..."
3,">я пью Святой ИсточникЭто не минеральная вода,...",Греча писча богов!,">Мне кажется, ОП - жирный тролльС хуя ли? Я са...",">>нет белков>Хорошая диета, правильная, неча с...","Мне кажется, ОП - жирный тролль",">С 28% слез оче быстро, на этих значениях перд...",">нет белковХорошая диета, правильная, неча сох...",Откуда у тебя рецепт на сибутрамин? Все аптеки...
4,">Покажи какие белки едят быки, гориллы, кенгур...",">я пью Святой ИсточникЭто не минеральная вода,...",Греча писча богов!,">Мне кажется, ОП - жирный тролльС хуя ли? Я са...",">>нет белков>Хорошая диета, правильная, неча с...","Мне кажется, ОП - жирный тролль",">С 28% слез оче быстро, на этих значениях перд...",">нет белковХорошая диета, правильная, неча сох..."


In [25]:
df = df.dropna()

In [26]:
trn_df, val_df = train_test_split(df, test_size = 0.1)
trn_df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
42279,"Да я самоучка , что за хедж мод ? пошёл смотреть",>улучшением данной страты является созданием 2...,"Не везёт пока с фючами , около 100 долларов сл...",Пошел нахуй,>ХОМЯКИ СОСНУТ! БИТКОИН ВЫШЕ 200 НЕ БУДЕТ!!@>Р...,"Так, забейте, обосрался в треде не автор по ит...",Быстрофикс 25к юсдт*,"Мне бы так жить, чтобы вкидываться по 25 юсдт ..."
43484,Зачем продавать? Пусть еще помайнят пару месяцев.,">Хочешь профит, научись гуглить, алладин.Почем...",Почему?,виталик нас кинул,"Эфир Скам, кидалик пидорас, майнинг не выгоден.",Есть шанс.,Как думаете эфир после перехода на pos стрельн...,"У меня щас рх574 на рейвене копает 3200р/мес, ..."
30973,*В какомФикс,А вкаком разделе надо было? Не ОП,"Не в том разделе создал, ну да ладно.>Могут ли...",Могут ли меня найти по скриншоту с рабочего ст...,В детстве при температуре видел пульсирующие ф...,"Оп, твоя фамилия не Калаби часом?","А мне как-то приснилось, как я посрал. Да, хор...",Как-то когда болел когда лежал на подушке я бу...
16438,Двачую,Двачую.,Запретить! Нипущать!,Двачую,В последнее время весь /b засрали фаги двух ти...,"Зашел ради интереса в каталог ньюсача, там из ...","Не стоит благодарности, я просто выполняю свою...",Про пидорах рабсийских
22526,https://www.youtube.com/watch?v=6uQDxLt94E8,"Хуя последователь Пичушкина тебе попался, лол",>засранныйА покажи мне чистые водоёмы кроме не...,"Про этих сомов в каждой жопе рассказывают, даж...",Хорошая статья. Про Радастею жуть пиздец. А пр...,>>703352,https://holod.media/2021/07/29/children/,Педофила понимаешь?) Если у деда стоит как в м...


In [27]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (1024 - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file):
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        
        r = (self.examples[item])
        r = [18 if v is None else v for v in r]
        try:
          return torch.tensor(r, dtype=torch.long)
        except:
          #print(r)
          return torch.tensor(r, dtype=torch.long)

In [28]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [29]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            
            loss = outputs['loss']  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir
    n_gpus = 1
    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1,n_gpus )
    # Note that DistributedSampler samples randomly
    device = 'cuda'
    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if n_gpus > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(device)
        labels = labels.to(device)
        try:
          with torch.no_grad():
              outputs = model(inputs, labels=labels)
              #print(outputs)
              lm_loss = outputs['loss']
              eval_loss += lm_loss.mean().item()
          nb_eval_steps += 1
        except:
          print(inputs)

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [30]:
from apex import amp

ModuleNotFoundError: ignored

In [31]:
cd apex

[Errno 2] No such file or directory: 'apex'
/content


In [32]:
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [39]:
from apex import amp

In [ ]:
main(trn_df, val_df)

10/10/2021 13:04:04 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
10/10/2021 13:04:04 - DEBUG - filelock -   Attempting to acquire lock 140301989053776 on cached/c73d5ea1365be5d1d2cc9f3ecb5c6afd77fbd85274871869ff81afeb6e5bbfd5.c7f979c8d927f0e43eb584070478140e210e1b8830f862d45e9f3d39eda2ef5d.lock
10/10/2021 13:04:04 - DEBUG - filelock -   Lock 140301989053776 acquired on cached/c73d5ea1365be5d1d2cc9f3ecb5c6afd77fbd85274871869ff81afeb6e5bbfd5.c7f979c8d927f0e43eb584070478140e210e1b8830f862d45e9f3d39eda2ef5d.lock


Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

10/10/2021 13:04:05 - DEBUG - filelock -   Attempting to release lock 140301989053776 on cached/c73d5ea1365be5d1d2cc9f3ecb5c6afd77fbd85274871869ff81afeb6e5bbfd5.c7f979c8d927f0e43eb584070478140e210e1b8830f862d45e9f3d39eda2ef5d.lock
10/10/2021 13:04:05 - DEBUG - filelock -   Lock 140301989053776 released on cached/c73d5ea1365be5d1d2cc9f3ecb5c6afd77fbd85274871869ff81afeb6e5bbfd5.c7f979c8d927f0e43eb584070478140e210e1b8830f862d45e9f3d39eda2ef5d.lock
10/10/2021 13:04:05 - DEBUG - filelock -   Attempting to acquire lock 140302047117072 on cached/ba89ad6b78f0cfd5099cf0c469a2efcdaf072a33eaf719eb1d9285ab6933a686.b488da21e063aa1d3ff664d686226d61ee451181ff9213f3da996b42d42f384b.lock
10/10/2021 13:04:05 - DEBUG - filelock -   Lock 140302047117072 acquired on cached/ba89ad6b78f0cfd5099cf0c469a2efcdaf072a33eaf719eb1d9285ab6933a686.b488da21e063aa1d3ff664d686226d61ee451181ff9213f3da996b42d42f384b.lock


Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

10/10/2021 13:04:06 - DEBUG - filelock -   Attempting to release lock 140302047117072 on cached/ba89ad6b78f0cfd5099cf0c469a2efcdaf072a33eaf719eb1d9285ab6933a686.b488da21e063aa1d3ff664d686226d61ee451181ff9213f3da996b42d42f384b.lock
10/10/2021 13:04:06 - DEBUG - filelock -   Lock 140302047117072 released on cached/ba89ad6b78f0cfd5099cf0c469a2efcdaf072a33eaf719eb1d9285ab6933a686.b488da21e063aa1d3ff664d686226d61ee451181ff9213f3da996b42d42f384b.lock
10/10/2021 13:04:06 - DEBUG - filelock -   Attempting to acquire lock 140302044322320 on cached/647ef91c1bb01f6b164a096f99ff5a7de7f1c429966df9dfbd3bc01ad42c7e5c.db5526f107eff4fd6c62a9a36a55cd29713e2ebbf3ffa34a9dc838d05f93e811.lock
10/10/2021 13:04:06 - DEBUG - filelock -   Lock 140302044322320 acquired on cached/647ef91c1bb01f6b164a096f99ff5a7de7f1c429966df9dfbd3bc01ad42c7e5c.db5526f107eff4fd6c62a9a36a55cd29713e2ebbf3ffa34a9dc838d05f93e811.lock


...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/42209 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
10/10/2021 13:30:05 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-7000
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
10/10/2021 13:30:18 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-7000
10/10/2021 13:53:43 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-14000
10/10/2021 13:53:55 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-14000
10/10/2021 14:17:21 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-21000
10/10/202

Iteration:   0%|          | 0/42209 [00:00<?, ?it/s]

10/10/2021 15:28:41 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-42000
10/10/2021 15:28:54 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-42000
10/10/2021 15:52:20 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-49000
10/10/2021 15:52:32 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-49000


In [26]:
!nvidia-smi

Sat Oct  9 20:14:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    34W / 250W |   2959MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2",from_tf=True)


def get_length_param(text: str) -> str:
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param


for step in range(5):
    
    input_user = input("===> User:")
    
    # encode the new user input, add parameters and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(f"|0|{get_length_param(input_user)}|" + input_user + tokenizer.eos_token +  "|1|1|", return_tensors="pt")

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    
    # generated a response
    chat_history_ids = model.generate(
        bot_input_ids,
        num_return_sequences=1,
        max_length=512,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature = 0.6,
        mask_token_id=tokenizer.mask_token_id,
        eos_token_id=tokenizer.eos_token_id,
        unk_token_id=tokenizer.unk_token_id,
        pad_token_id=tokenizer.pad_token_id,
        device='cpu',
    )
    
    # pretty print last ouput tokens from bot
    print(f"===> RuDialoGPT: {tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)}")

KeyboardInterrupt: ignored

In [35]:
pip install transformers==4.4.2

     |████████████████████████████████| 2.0 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 62.4 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.0
    Uninstalling transformers-3.5.0:
      Successfully uninstalled transformers-3.5.0


In [25]:
!rm -r /content/apex/output-small/checkpoint*

In [ ]:
checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )

In [ ]:
def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir
    n_gpus = 1
    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1,n_gpus )
    # Note that DistributedSampler samples randomly
    device = 'cuda'
    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if n_gpus > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(device)
        labels = labels.to(device)
        try:
          with torch.no_grad():
              outputs = model(inputs, labels=labels[:inputs.size()])
              lm_loss = outputs[0]
              eval_loss += lm_loss.mean().item()
          nb_eval_steps += 1
        except:
          print(inputs)

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}
    print(result)
    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
 config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
 tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)

 results = {}
 for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained('/content/output-small')
            model.to('cuda')
            
            result = evaluate(args, model, tokenizer, trn_df, val_df, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            print(result)
            results.update(result)
            
            
            

/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


RuntimeError: ignored

In [ ]:
results

{}

In [ ]:
!pip install apex

  Using cached apex-0.9.10.dev0-py3-none-any.whl


In [ ]:
!sh setup.sh

In [ ]:
!nvidia-smi

Sat Sep 25 19:49:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    33W / 250W |  16043MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --fp16_opt_level 'O4' \
    --num_train_epochs 5 \
    --block_size 512 \
    --overwrite_output_dir

09/25/2021 19:38:49 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
09/25/2021 19:38:57 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-08, block_size=512, cache_dir=None, config_name=None, device=device(type='cuda'), do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='valid.txt',

In [ ]:
 from apex import amp

ImportError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(backbone,padding=True' 'truncation=True)
model = AutoModelWithLMHead.from_pretrained('output-small')
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)
# append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
# generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature = 0.8
    )
    
    # pretty print last ouput tokens from bot
    print("2c: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


привет


ValueError: ignored

In [ ]:
/content/output-small

In [27]:
!zip -r 2chAI_bot.zip output-small

  adding: output-small/ (stored 0%)
  adding: output-small/vocab.json (deflated 74%)
  adding: output-small/added_tokens.json (stored 0%)
  adding: output-small/pytorch_model.bin (deflated 100%)
  adding: output-small/special_tokens_map.json (deflated 72%)
  adding: output-small/tokenizer_config.json (deflated 66%)
  adding: output-small/config.json (deflated 50%)
  adding: output-small/training_args.bin (deflated 43%)
  adding: output-small/merges.txt (deflated 76%)


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
